In [1]:
import requests
import pandas as pd
import time
import datetime

#https://stackoverflow.com/questions/49188960/how-to-show-all-columns-names-on-a-large-pandas-dataframe
pd.set_option('display.max_columns', None)

In [2]:
#additional sources: (for readme or w/e)
#https://www.epochconverter.com/

In [3]:
#duilding starting dataframe for first subreddit:
subreddit = 'dndnext'
start_date = '1635356099'
kind = 'comment'

stem = 'https://api.pushshift.io'
slug = f'/reddit/search/{kind}/?subreddit={subreddit}&before={start_date}&size=100'

In [4]:
res = requests.get(stem + slug)

In [5]:
res

<Response [200]>

In [6]:
data = res.json()

In [7]:
#data['data']
#comments

In [8]:
#commented out for length
data['data'][0]

{'all_awardings': [],
 'archived': False,
 'associated_award': None,
 'author': 'DMPatrick',
 'author_flair_background_color': None,
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_template_id': None,
 'author_flair_text': None,
 'author_flair_text_color': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_bs9xa6ae',
 'author_patreon_flair': False,
 'author_premium': False,
 'body': "Well first of all, if that's the only way you find enjoyment then that's okay. You can play all of your characters that way if that's how you have fun.\n\nBut a few thoughts and suggestions that might help... \n\nFill out the traits, bonds, flaws, and ideals on your character sheets and refer back to them regularly.  If you're playing a character who shoots first and asks questions later, write that down! It helps to have some writen words to keep you in character.\n\nPerhaps a moot suggestion but try using a voice or accent. This will help mentally separate the chara

In [9]:
posts = []

for post in data['data']:
    post_dict = {
        'subreddit': post['subreddit'],
        'author': post['author'],
        'body': post['body'],
        'created_utc': post['created_utc']
                }
    posts.append(post_dict)

In [10]:
pd.DataFrame(posts).head(3)

,subreddit,author,body,created_utc
0,dndnext,DMPatrick,"Well first of all, if that's the only way you ...",1635356097
1,dndnext,whitetempest521,The feat is where almost the entire power budg...,1635356068
2,dndnext,Th1nker26,Oh I don't think it's a problem. I thought the...,1635356043


In [11]:
pd.DataFrame(posts)['created_utc'].min()

1635355058

In [12]:
pd.DataFrame(posts).to_csv('./csv_files/dndnext_comm.csv', index = False)

- Manually building dataframe for second subreddit:

In [13]:
subreddit_1 = 'dmacademy'

stem = 'https://api.pushshift.io'
slug_1 = f'/reddit/search/{kind}/?subreddit={subreddit_1}&before={start_date}&size=100'

In [14]:
res_1 = requests.get(stem + slug_1)

In [15]:
res_1

<Response [200]>

In [16]:
data_1 = res_1.json()

In [17]:
posts_1 = []

for post in data_1['data']:
    post_dict = {
        'subreddit': post['subreddit'],
        'author': post['author'],
        'body': post['body'],
        'created_utc': post['created_utc']
                }
    posts_1.append(post_dict)

In [18]:
pd.DataFrame(posts_1).head(3)

,subreddit,author,body,created_utc
0,DMAcademy,jordanleveledup,Just give him a fever. Sweats. Upset stomach. ...,1635356025
1,DMAcademy,YUSOBADDOG,"Firstly, obviously your not that good at impro...",1635355991
2,DMAcademy,FlexterityCheck,&gt;You can use their interest in the town to ...,1635355937


In [19]:
pd.DataFrame(posts_1)['created_utc'].min()

1635353637

In [20]:
pd.DataFrame(posts_1).to_csv('./csv_files/dmacademy_comm.csv', index = False)

In [21]:
#copied from Terri John
def pushshift(subreddit, kind = 'comment', start_date = '1635356099',
              iters = 10, og_csv = './csv_files/dndnext.csv'):
    
    for scrape in range(1,iters + 1):
        date = pd.read_csv(og_csv)['created_utc'].min()
        stem = 'https://api.pushshift.io'
        slug = f'/reddit/search/{kind}/?subreddit={subreddit}&before={date}&size=100&selftext=true'
        
        res = requests.get(stem + slug) 
        
        if res.status_code != 200:
            print(f'url: {stem + slug} returned {res.status_code}')
            
        else:
            data = res.json()
            posts = []

            for post in data['data']:
                post_dict = {
                    'subreddit' : post['subreddit'],
                    'body' : post['body'],
                    'author' : post['author'],
                    'created_utc' : post['created_utc']
                }
                posts.append(post_dict)
                
                
            og_df = pd.read_csv(og_csv)
            temp_df = pd.DataFrame(posts)
            
            new_df = pd.concat([og_df,temp_df])
            
            new_df.to_csv(og_csv,index=False)
            
            print('Scrape success!')
            print(f'Original data was {og_df.shape[0]} rows')
            print(f'New data is {new_df.shape[0]} rows!')
            print(new_df['created_utc'].min())
            time.sleep(3)

In [27]:
#pushshift(subreddit = 'dndnext', kind = 'comment', start_date = '1635356099',
#              iters = 199, og_csv = './csv_files/dndnext_comm.csv')

#commented out for length

In [28]:
#pushshift(subreddit = 'dmacademy', kind = 'comment', start_date = '1635356099',
#          iters = 199, og_csv = './csv_files/dmacademy_comm.csv')

#commented out for length

In [24]:
dndnext_subs_df = pd.read_csv('./csv_files/dndnext_comm.csv')
dndnext_subs_df.head(3)

,subreddit,author,body,created_utc
0,dndnext,DMPatrick,"Well first of all, if that's the only way you ...",1635356097
1,dndnext,whitetempest521,The feat is where almost the entire power budg...,1635356068
2,dndnext,Th1nker26,Oh I don't think it's a problem. I thought the...,1635356043


In [25]:
dmacademy_subs_df = pd.read_csv('./csv_files/dmacademy_comm.csv')
dmacademy_subs_df.head(3)

,subreddit,author,body,created_utc
0,DMAcademy,jordanleveledup,Just give him a fever. Sweats. Upset stomach. ...,1635356025
1,DMAcademy,YUSOBADDOG,"Firstly, obviously your not that good at impro...",1635355991
2,DMAcademy,FlexterityCheck,&gt;You can use their interest in the town to ...,1635355937


In [26]:
df = pd.concat([dndnext_subs_df, dmacademy_subs_df], ignore_index = True)
df.to_csv('./csv_files/comms_orginal.csv', index=False)